# Introduction to Gradio

Code based on the [official documentation](https://www.gradio.app/docs/gradio/blocks) of Gradio

_Written by_: Enrique Noriega

_Last edited_: 10/05/2024

Set up gradio and dependencies

In [2]:
%pip install gradio==4.44.1 #https://gradio-builds.s3.amazonaws.com/a0c487cd57a217775f0d1bc77c041b7cd516cc8a/gradio-3.41.2-py3-none-any.whl

  Using cached aiofiles-23.2.1-py3-none-any.whl.metadata (9.7 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached tomlkit-0.12.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached websockets-12.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.6 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 63.6 MB/s eta 0:00:00a 0:00:01
Using cached tomlkit-0.12.0-py3-none-any.whl (37 kB)
Using cached aiofiles-23.2.1-py3-none-any.whl (15 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 70.5 MB/s eta 0:00:00
Using cached semantic_version-2.10.0-py2.py3-none-any.whl (15 kB)
Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)
Using cached shellingham-1.5.4-py2.py3-none-any.whl (9.8 kB)
Using cached websockets-12.0-cp311-cp311-macosx_11_0_arm64.whl (121 kB)
Note: you may need to restart the kernel to

In [11]:
%pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 9.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 10.4 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


## Interface API

Gradio is a rapid prototype UI library for python. It is designed to be able to quickly and easily build machine learning application demos, but it will work for any other purpose as well.

The development _model_ of Gradio is to provide __Interface__ to an arbitrary python function.

Consider the following toy function below.

In [66]:
def greet(name):
    """ This is a sample hello world function """

    return f"Greetings, {name} "

This function takes one `string` argument, and returns a greeting message.

With Gradio, we can create a __GUI__ around it, let's do it.

Load the gradio jupyter extension for this demo

In [4]:
%load_ext gradio

In [5]:
import gradio as gr

In [67]:
%%blocks
# This is a handy magic command

# Our interface  wraps the greet function

demo = gr.Interface(
    fn=greet,             # Here, we are wiring the function to the interface
    inputs=["text"],      # Specify the input types
    outputs=["text"],     # Same, for output
)


Observe how we have web controls rendered for the input and output, and all the "wiring" is done for us.

There are more control types.

In [7]:
%%blocks

def greet(name, intensity):
    return "Hello, " + name + "!" * intensity

demo = gr.Interface(
    fn=greet,
    inputs=["text", gr.Slider(value=2, minimum=1, maximum=10, step=1)],
    outputs=[gr.Textbox(label="greeting", lines=3)],
)



Suppose you had a more complex function, with multiple outputs as well. In the example below, we define a function that takes a string, boolean, and number, and returns a string and number.

In [8]:
%%blocks

def greet(name, is_morning, temperature):
    salutation = "Good morning" if is_morning else "Good evening"
    greeting = f"{salutation} {name}. It is {temperature} degrees today"
    celsius = (temperature - 32) * 5 / 9
    return greeting, round(celsius, 2)

demo = gr.Interface(
    fn=greet,
    inputs=["text", "checkbox", gr.Slider(0, 100)],
    outputs=["text", "number"],
)

Gradio supports many types of components, such as Image, DataFrame, Video, or Label. Let's try an image-to-image function to get a feel for these!

In [9]:

import numpy as np

def sepia(input_img):
    print("Hola")
    sepia_filter = np.array([
        [0.393, 0.769, 0.189],
        [0.349, 0.686, 0.168],
        [0.272, 0.534, 0.131]
    ])
    sepia_img = input_img.dot(sepia_filter.T)
    sepia_img /= sepia_img.max()
    return sepia_img

demo = gr.Interface(sepia, gr.Image(), "image")
demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Hola


You can provide example data that a user can easily load into Interface. This can be helpful to demonstrate the types of inputs the model expects, as well as to provide a way to explore your dataset in conjunction with your model.

In [10]:
%%blocks

def calculator(num1, operation, num2):
    if operation == "add":
        return num1 + num2
    elif operation == "subtract":
        return num1 - num2
    elif operation == "multiply":
        return num1 * num2
    elif operation == "divide":
        if num2 == 0:
            raise gr.Error("Cannot divide by zero!")
        return num1 / num2

demo = gr.Interface(
    calculator,
    [
        "number",
        gr.Radio(["add", "subtract", "multiply", "divide"]),
        "number"
    ],
    "number",
    examples=[
        [45, "add", 3],
        [3.14, "divide", 2],
        [144, "multiply", 2.5],
        [0, "subtract", 1.2],
    ],
    title="Toy Calculator",
    description="Here's a sample toy calculator.",
)

Now, let's actually use _machine learning_ and create an interface

In [12]:
from transformers import pipeline

In [13]:
classifier = pipeline("text-classification")

def classify(input):
  results = classifier(input)
  return results[0]['label'], results[0]['score']

classify("This movie was awesome")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


('POSITIVE', 0.9998660087585449)

In [14]:
%%blocks

demo = gr.Interface(
    classify,
    inputs=[
        "text"
    ],
    outputs=["text","number"],

    title="DistillBERT classifier",
    description="Type some text, we will tell you whether its positive or negative",
)

## Blocks API

When you need finer grained control of the layout and the components, you can use the _Blocks_ api instead of the interface.

With the blocks api, you declaratively define which controls you are going to display and it is your responsability as developer to connect each control with the corresponding inputs and outputs

In [15]:
%%blocks

def update(name):
    return f"Welcome to Gradio, {name}!"

with gr.Blocks() as demo:
    gr.Markdown("Start typing below and then click **Run** to see the output.")
    with gr.Row():
        inp = gr.Textbox(placeholder="What is your name?")
        out = gr.Textbox()
    btn = gr.Button("Run")
    btn.click(fn=update, inputs=inp, outputs=out)

You can see how we have precise control over the control types, their positioning on the UI and the binding of the events.

## Chat Interface

A very popular use case for Gradio is building chat bot applications. Gradio contains a pre-built component that handles the standard features of a chatbot, ready to be connected to your LLM of choice via your prefered programming framework.

Let's take a look at the `ChatInterface` control.

In [19]:
%%blocks

def yes(message, history):
    return "yes"

def vote(data: gr.LikeData):
    if data.liked:
        print("You upvoted this response: " + data.value)
    else:
        print("You downvoted this response: " + data.value)

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(placeholder="<strong>Your Personal Yes-Man</strong><br>Ask Me Anything")
    chatbot.like(vote, None, None)
    gr.ChatInterface(fn=yes, chatbot=chatbot)

You upvoted this response: yes
You downvoted this response: yes
You upvoted this response: yes


Let's build a real chatbot using Verde AI

In [22]:
%pip install langchain langchain_core langchain_openai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [63]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from itertools import chain as itchain

import os
# os.environ["OPENAI_API_KEY"] = "..."

API_ENDPOINT = "https://llm1.cyverse.ai/v1"

llm = ChatOpenAI(model="Mistral-7B-Instruct-v0.3", base_url=API_ENDPOINT)
conversation_template = ChatPromptTemplate([
    ("system", "Speak like Super Mario Bros. You're in a friendly conversation with a gamer"),
    ("placeholder", "{conversation}"),
    ("human", "{message}")
])

chain = conversation_template | llm | StrOutputParser()

def send_message(message, history):
    history = list(itchain.from_iterable([("human", human), ("ai", ai)] for human, ai in history))

    return chain.invoke({"conversation":history, "message":message})

In [64]:
%%blocks


def vote(data: gr.LikeData):
    if data.liked:
        print("You upvoted this response: " + data.value)
    else:
        print("You downvoted this response: " + data.value)

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(placeholder="<strong>Your Personal Yes-Man</strong><br>Ask Me Anything")
    chatbot.like(vote, None, None)
    gr.ChatInterface(fn=send_message, chatbot=chatbot)

## Additional features

You can easily [deploy your gradio app in HuggingFace 🤗 spaces](https://huggingface.co/docs/hub/en/spaces-sdks-gradio) for free to showcase your demo.
You can also do [client side code using javascript](https://www.gradio.app/guides/custom-CSS-and-JS) and code your custom controls to extend Gradio's functionality.

Have fun!